In [1]:
import requests
import json
import pandas as pd
from models import *

2018-11-14 13:45:34,137 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-11-14 13:45:34,138 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,139 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-11-14 13:45:34,140 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,142 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("events")
2018-11-14 13:45:34,142 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,145 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("brands")
2018-11-14 13:45:34,146 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,148 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("brands_events")
2018-11-14 13:45:34,149 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,150 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("reactions")
2018-11-14 13:45:34,151 INFO sqlalchemy.engine.base.Engine ()
2018-11-14 13:45:34,152 INFO sql

In [2]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class Results: 
    
    def get_results(date,events):
        data = []
        for i in range(0, events, 100):
            url = 'https://api.fda.gov/drug/event.json?api_key=iQLpf10vDEd4qDbJfIXK9Tm9SYFXo2cCoy4Ghv62&search=receivedate:' + str(date) + '&limit=100'+'&skip='+str(i)
            r = requests.get(url)
            jsoning_data = r.json()
            data += list(jsoning_data['results'])
        return data

In [7]:
class Instantiating_Holidays: 
    
    def holidays(event): #change to holiday_json later
            if event['receiptdate'] == '20171231':
                holiday = 'New Years Eve'
            elif event['receiptdate'] == '20171031':
                holiday = 'Halloween'
            elif event['receiptdate'] == '20170214':
                holiday = 'Valentine\'s Day'
            elif event['receiptdate'] == '20170228':
                holiday = 'Mardi Gras'
#             elif event['receiptdate'] == '20170317':
#                 holiday = 'St. Patrick\'s Day'
            elif event['receiptdate'] == '20170420':
                holiday = 'Cannabis Day'
            elif event['receiptdate'] == '20170505':
                holiday = 'Cinco de Mayo'
            elif event['receiptdate'] == '20170704':
                holiday = 'Independence Day'
            elif event['receiptdate'] == '20171123':
                holiday = 'Thanksgiving'
            elif event['receiptdate'] == '20171225':
                holiday = 'Christmas'
                
            holiday_name = Holidays(name=holiday,date=str(event['receiptdate']))
            return holiday_name

In [6]:
sp = Results.get_results(20170317, 100)[0]
sp['receiptdate']

'20170705'

In [8]:
## GLOBAL FUNCTIONS

def sex(event):
    patients = []
    sex = []
    if 'patient' in event:
        sex.append(event['patient'])
    else: 
        sex.append('None')
    for i in sex:
        if 'patientsex' in i: 
            patients.append(int(i['patientsex']))
        else: 
            patients.append("None")
    return patients[0]

    
def age(event):
    patients = []
    ages = []
#     if 'patient' in event:
#         ages.append(event['patient'])
#     else: 
#         ages.append('None')
#     for i in ages:
#         if 'patientonsetage' in i: 
#             patients.append(int(i['patientonsetage']))
#         else: 
#             patients.append('None')
#     return patients[0]
    ages_list = []
    if 'patient' in event:
        ages.append(event['patient'])
    else: 
        ages.append('None')
    for i in ages:
        if 'patientonsetage' in i: 
            patients.append(i['patientonsetage'])
        else: 
            patients.append(0)
    for x in patients:
        if x == None:
            ages_list.append(str(x))
        else:
            ages_list.append(int(x))

    return ages_list[0]

In [9]:
class Instantiating_Events:

    def event_object(date,events):
        holiday_json = Results.get_results(date,events)
        our_holiday = Instantiating_Holidays.holidays(holiday_json[0])

        for event in holiday_json:
            sex_func = sex(event) 
            age_func = age(event)
            new_event = Adverse_Events(sex=sex_func, age=age_func, holidays = our_holiday)
            
            
            b = []
            if 'openfda' in event['patient']['drug'][0]:
                    if 'brand_name' in event['patient']['drug'][0]['openfda']:
                        b.extend(event['patient']['drug'][0]['openfda']['brand_name'])
                    else:
                        b.extend('None')
                        
            brand_list = []
            for brand in b: 
                brand_result = session.query(Brands).filter(Brands.name == brand).first()
                if brand_result: 
                    brand_list.append(brand_result)
                else: 
                    brand_list.append(Brands(name=brand))
            new_event.brands = brand_list
            
            
            
        
            reactions =  event['patient']['reaction']
            list_of_reactions = []
            for reaction in reactions: 
                list_of_reactions.append(reaction['reactionmeddrapt'])
            
            reaction_list = []
            for reaction in list_of_reactions: 
                reaction_result = session.query(Reactions).filter(Reactions.name == reaction).first()
                if reaction_result: 
                    reaction_list.append(reaction_result)
                else: 
                    reaction_list.append(Reactions(name=reaction))
            new_event.reactions = list(set(reaction_list))
            
            
            session.add(new_event)
            session.commit()

In [17]:
Instantiating_Events.event_object(20171231,400)

2018-11-14 14:04:47,283 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 14:04:47,284 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:47,285 INFO sqlalchemy.engine.base.Engine ('CLOPIDOGREL BISULFATE', 1, 0)
2018-11-14 14:04:47,287 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:47,288 INFO sqlalchemy.engine.base.Engine ('CLOPIDOGREL', 1, 0)
2018-11-14 14:04:47,291 INFO sqlalchemy.engine.base.Engine INSERT INTO holidays (name, date) VALUES (?, ?)
2018-11-14 14:04:47,292 INFO sqlalchemy.engine.base.Engine ('New Years Eve', '20171231')
2018-11-14 14:04:47,293 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 14:04:47,296 INFO sqlalchemy.engine.base.Engine (1, 75, 9)
2018-11-14 14:04:47,299 IN

2018-11-14 14:04:47,530 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:47,532 INFO sqlalchemy.engine.base.Engine ('Drug administered at inappropriate site', 1, 0)
2018-11-14 14:04:47,534 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:47,536 INFO sqlalchemy.engine.base.Engine ('Injection site swelling', 1, 0)
2018-11-14 14:04:47,538 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions, reactions_events 
WHERE ? = reactions_events.event_id AND reactions.id = reactions_events.reactions_id
2018-11-14 14:04:47,545 INFO sqlalchemy.engine.base.Engine (28402,)
2018-11-14 14:04:47,557 INFO sqlalchemy.engine.base.Engine INSERT INTO reactions_events (reactions_id, 

2018-11-14 14:04:47,780 INFO sqlalchemy.engine.base.Engine ((104, 28405), (954, 28405))
2018-11-14 14:04:47,782 INFO sqlalchemy.engine.base.Engine COMMIT
2018-11-14 14:04:47,796 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 14:04:47,798 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 14:04:47,802 INFO sqlalchemy.engine.base.Engine (9,)
2018-11-14 14:04:47,805 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 14:04:47,813 INFO sqlalchemy.engine.base.Engine (2, 0, 9)
2018-11-14 14:04:47,830 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:47,851 INFO sqlalchemy.engine.base.Engine ('COSENTYX', 1, 0)
2018-11-14 14:04:47,855 INFO sqlalchemy.engine.base.Engine SELECT brands.id

2018-11-14 14:04:48,173 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:48,174 INFO sqlalchemy.engine.base.Engine ('COSENTYX', 1, 0)
2018-11-14 14:04:48,178 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 14:04:48,179 INFO sqlalchemy.engine.base.Engine (28409,)
2018-11-14 14:04:48,192 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 14:04:48,196 INFO sqlalchemy.engine.base.Engine (219, 28409)
2018-11-14 14:04:48,202 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:48,204 INFO sqlalchemy.engine.base.Engine ('Confusional state', 1, 

2018-11-14 14:04:48,472 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 14:04:48,473 INFO sqlalchemy.engine.base.Engine (9,)
2018-11-14 14:04:48,475 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 14:04:48,476 INFO sqlalchemy.engine.base.Engine (1, 52, 9)
2018-11-14 14:04:48,479 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:04:48,480 INFO sqlalchemy.engine.base.Engine ('COSENTYX', 1, 0)
2018-11-14 14:04:48,482 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 14:04:48,484 INFO sqlalchemy.engine.base.Engine (28413,)
2018-11-14 14:04:4

...

2018-11-14 14:05:19,299 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:19,301 INFO sqlalchemy.engine.base.Engine ('Encephalopathy', 1, 0)
2018-11-14 14:05:19,306 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:19,307 INFO sqlalchemy.engine.base.Engine ('Chronic obstructive pulmonary disease', 1, 0)
2018-11-14 14:05:19,309 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:19,311 INFO sqlalchemy.engine.base.Engine ('Delirium', 1, 0)
2018-11-14 14:05:19,313 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.na

2018-11-14 14:05:19,530 INFO sqlalchemy.engine.base.Engine ('ORENCIA', 1, 0)
2018-11-14 14:05:19,532 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 14:05:19,534 INFO sqlalchemy.engine.base.Engine (28736,)
2018-11-14 14:05:19,549 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 14:05:19,550 INFO sqlalchemy.engine.base.Engine (2215, 28736)
2018-11-14 14:05:19,574 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:19,576 INFO sqlalchemy.engine.base.Engine ('Bedridden', 1, 0)
2018-11-14 14:05:19,590 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMI

2018-11-14 14:05:19,817 INFO sqlalchemy.engine.base.Engine ((17, 28739), (364, 28739))
2018-11-14 14:05:19,819 INFO sqlalchemy.engine.base.Engine COMMIT
2018-11-14 14:05:19,826 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-11-14 14:05:19,828 INFO sqlalchemy.engine.base.Engine SELECT holidays.id AS holidays_id, holidays.name AS holidays_name, holidays.date AS holidays_date 
FROM holidays 
WHERE holidays.id = ?
2018-11-14 14:05:19,829 INFO sqlalchemy.engine.base.Engine (9,)
2018-11-14 14:05:19,833 INFO sqlalchemy.engine.base.Engine INSERT INTO events (sex, age, holiday_id) VALUES (?, ?, ?)
2018-11-14 14:05:19,834 INFO sqlalchemy.engine.base.Engine (2, 89, 9)
2018-11-14 14:05:19,839 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:19,841 INFO sqlalchemy.engine.base.Engine ('ADCIRCA', 1, 0)
2018-11-14 14:05:19,844 INFO sqlalchemy.engine.base.Engine SELECT brands.id 

2018-11-14 14:05:20,057 INFO sqlalchemy.engine.base.Engine ('ACUNIVIVE 90',)
2018-11-14 14:05:20,059 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 14:05:20,060 INFO sqlalchemy.engine.base.Engine ((6755, 28741), (6756, 28741), (6759, 28741), (6758, 28741), (6751, 28741), (6752, 28741), (6754, 28741), (6757, 28741), (6753, 28741))
2018-11-14 14:05:20,062 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:20,063 INFO sqlalchemy.engine.base.Engine ('Seizure', 1, 0)
2018-11-14 14:05:20,065 INFO sqlalchemy.engine.base.Engine SELECT reactions.id AS reactions_id, reactions.name AS reactions_name 
FROM reactions 
WHERE reactions.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:20,066 INFO sqlalchemy.engine.base.Engine ('Syncope', 1, 0)
2018-11-14 14:05:20,068 INFO sqlalchemy.engine.base.Engine SELECT reactions.

2018-11-14 14:05:20,345 INFO sqlalchemy.engine.base.Engine ('ZESTRIL', 1, 0)
2018-11-14 14:05:20,350 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:20,354 INFO sqlalchemy.engine.base.Engine ('QBRELIS', 1, 0)
2018-11-14 14:05:20,357 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands 
WHERE brands.name = ?
 LIMIT ? OFFSET ?
2018-11-14 14:05:20,358 INFO sqlalchemy.engine.base.Engine ('LISINOPRIL', 1, 0)
2018-11-14 14:05:20,360 INFO sqlalchemy.engine.base.Engine SELECT brands.id AS brands_id, brands.name AS brands_name 
FROM brands, brands_events 
WHERE ? = brands_events.event_id AND brands.id = brands_events.brand_id
2018-11-14 14:05:20,362 INFO sqlalchemy.engine.base.Engine (28745,)
2018-11-14 14:05:20,373 INFO sqlalchemy.engine.base.Engine INSERT INTO brands_events (brand_id, event_id) VALUES (?, ?)
2018-11-14 14:05:2

In [14]:
cinco_mayo = Results.get_results(20170214,4100)

In [50]:
def age_new(event):
    patients = []
    ages_list = []
    for i in event:
        ages = []
        if 'patient' in i:
            ages.append(i['patient'])
        else: 
            ages.append('None')
        for i in ages:
            if 'patientonsetage' in i: 
                patients.append(i['patientonsetage'])
            else: 
                patients.append(0)
        for x in patients:
            if x == None:
                ages_list.append(str(x))
            else:
                ages_list.append(int(x))
                
        return ages_list[0]
#             else:
#                 patients.append('None')
#     return patients

In [14]:
list_num = [23, 222, 33, 444, 5555, 23]
def remove_num(numbers):
    new_list = []
    for i in numbers:
        if i > 200:
            del i
        else:
            new_list.append(i)
            
            
    return new_list


In [15]:
remove_num(list_num)

[23, 33, 23]